Setup PyTorch to use best hardware option

In [24]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

mps


Get model and tokenizer from Hugginface

In [25]:
from transformers import AutoTokenizer, AutoModel

model_name = "distilbert/distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, clean_up_tokenization_spaces=False, use_fast=True)
model = AutoModel.from_pretrained(model_name).to(device)
model.eval()

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-5): 6 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout)

Dataset load

In [26]:
ARTIFACTS_BASE = '../../../artifacts'

In [28]:
from os import path
from datasets import load_from_disk

dataset_path = path.join(ARTIFACTS_BASE, 'step-1-classic-ml', 'distilroberta-base')

train_dataset = load_from_disk(path.join(dataset_path, 'train'))
test_dataset = load_from_disk(path.join(dataset_path, 'test'))

Prepare loaded dataset for metrics counting

In [29]:
import numpy as np

X_test_embedding_list = np.array(test_dataset["embedding"])
y_test_label_list = np.array(test_dataset["label"])

X_train_embedding_list = np.array(train_dataset["embedding"])
y_train_label_list = np.array(train_dataset["label"])

In [30]:
# Import classification models
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.neural_network import MLPClassifier

# Initialize estimators using their default parameters
estimators = [
    ("Naive Bayes", GaussianNB()),
    ("LGB Classifier", lgb.LGBMClassifier()),
    ("Random Forest", RandomForestClassifier(n_estimators=100, max_depth=10, n_jobs=-1)),
    ("MLPClassifier", MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', random_state=42)),
]

In [31]:
# Import performance metrics libraries
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# Prepare a DataFrame to keep track of the models' performance
results = pd.DataFrame(columns=["accuracy", "precision", "recall", "f1 score"])

# Iterate through each estimator in the list
for est_name, est_obj in tqdm(desc='Calculate metrics', iterable=estimators):

    if hasattr(est_obj, 'to'):
        est_obj.to(device)  # Move model to GPU if supported
    
    # Fit the model
    est_obj.fit(X_train_embedding_list, y_train_label_list)
    
    # Use the model to predict unseen prompts
    y_predict = est_obj.predict(X_test_embedding_list)
    
    # Calculate performance metrics
    accuracy = accuracy_score(y_test_label_list, y_predict)
    precision = precision_score(y_test_label_list, y_predict)
    recall = recall_score(y_test_label_list, y_predict)
    f1 = f1_score(y_test_label_list, y_predict)  
    
    # Store performance metrics
    results.loc[est_name] = [accuracy, precision, recall, f1]

Calculate metrics:  25%|██▌       | 1/4 [00:01<00:05,  1.78s/it]

[LightGBM] [Info] Number of positive: 129232, number of negative: 132506
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.372230 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195840
[LightGBM] [Info] Number of data points in the train set: 261738, number of used features: 768
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.493746 -> initscore=-0.025019
[LightGBM] [Info] Start training from score -0.025019


Calculate metrics: 100%|██████████| 4/4 [05:04<00:00, 76.14s/it] 


In [32]:
results

,accuracy,precision,recall,f1 score
Naive Bayes,0.827886,0.817184,0.839072,0.827983
LGB Classifier,0.940580,0.941445,0.937976,0.939707
Random Forest,0.910190,0.904418,0.914752,0.909556
MLPClassifier,0.980968,0.974480,0.987304,0.980850
